In [11]:
from core.utils import Tibanna
from dcicutils import ff_utils
from core.utils import run_workflow
from datetime import datetime
from core.wfr import *

env = 'data'
tibanna = Tibanna(env=env)
ff = ff_utils.fdn_connection(key={"default" : tibanna.ff_keys})


In [10]:
import time

# for a given experiment set and some parameters like instrument
# print set of files and their partA hic workflow status
# if there are one that are running report the number of running cases
# if there are file pairs that don't have a corresponding part A, report them separately

add_wfr = True
# url for hic exps
pairs_url = "/search/?file_format=pairs&file_type=contact+list-replicate&file_type=contact+list-combined"+ \
            "&status=released&status=released+to+project&status=uploaded&type=FileProcessed" + \
            "&quality_metric.uuid=No+value&limit=all"
pairs_files = ff_utils.search_metadata(pairs_url , ff_env=env)
counter = 0
completed = 0
print len(pairs_files)

okay = 0
non = 0
running = 0
for a_pairs in pairs_files:
    if a_pairs['accession'] in ['4DNFIF2WIXVB','4DNFIZDCXIL7','4DNFI2WWXWR3','4DNFI21EIR1D']:
        #report = get_wfr_out(a_pairs['accession'], 'pairsqc-single 0.2.5', None, env)
        if add_wfr:
            # get experiment info
            exp_acc = a_pairs.get('source_experiments')
            
            if not exp_acc:
                print(a_pairs['accession'], 'does not have source experiments, skipping')
                continue
            
            exp_type, chrsize = extract_no_nz_file(exp_acc[0], env)
            
            if exp_type == 'skip':
                print 'skipping'
                continue
#     

            if exp_type in ['CHIA-pet', 'TrAC-loop','micro-C']:
                print 'skipping', exp_type
                continue
                
            parameters = {"enzyme": '4',
                      "sample_name": a_pairs['accession']} 

            run_guide = {'wf_name': 'pairsqc-single',
                         'wf_uuid': 'ae3a87cb-3fa2-469e-97c7-540fc2d0a117',
                         'parameters': parameters}

            inp_f = {'input_pairs':a_pairs['accession'], 'chromsizes':chrsize} 
            print(a_pairs['accession'], 'starting the run', exp_type)
            run_missing_wfr(run_guide, inp_f, a_pairs['accession']+'_4_noNZ_DnaseHiC', env, tibanna)   
            break


#         else:
#             print(a_pairs['accession'], 'needs a run')
# #     elif report == 'running':
# #         running += 1
# #         print(a_pairs['accession'], 'still running')
# #     else:
# #         okay += 1
     
print("okay, not okay, running")    
print(okay, non, running)


119
skipping CHIA-pet
skipping TrAC-loop
skipping micro-C
(u'4DNFI21EIR1D', 'starting the run', u'DNase Hi-C')
about to start run pairsqc-single_4DNFI21EIR1D_4_noNZ_DnaseHiC
response from aws was: 
 {u'startDate': datetime.datetime(2018, 6, 5, 16, 8, 38, 19000, tzinfo=tzlocal()), 'ResponseMetadata': {'RetryAttempts': 0, 'HTTPStatusCode': 200, 'RequestId': '3bdaea94-68fc-11e8-92c1-cd8546da3d62', 'HTTPHeaders': {'x-amzn-requestid': '3bdaea94-68fc-11e8-92c1-cd8546da3d62', 'content-length': '152', 'content-type': 'application/x-amz-json-1.0'}}, u'executionArn': u'arn:aws:states:us-east-1:643366669028:execution:tibanna_pony:pairsqc-single_4DNFI21EIR1D_4_noNZ_DnaseHiC'}
url to view status:
https://console.aws.amazon.com/states/home?region=us-east-1#/executions/details/arn:aws:states:us-east-1:643366669028:execution:tibanna_pony:pairsqc-single_4DNFI21EIR1D_4_noNZ_DnaseHiC


KeyboardInterrupt: 

In [ ]:
# Release pairs qc

pairs_url = '/search/?file_format=pairs&status=released&status=released%20to%20project&status=uploaded&type=FileProcessed'
pairs_files = ff_utils.get_metadata(pairs_url , connection=ff)['@graph']
pairs_files = [i for i in pairs_files if i.get('quality_metric')]
print len(pairs_files)
success = 0
patch_problem = 0

for pairs in pairs_files:
    pairs_status = pairs['status']
    if pairs_status in ['released', 'released to project']:
        qc_status = ff_utils.get_metadata(pairs['quality_metric'],connection=ff)['status']
        if qc_status != pairs_status:
            patch_data = {"status": pairs_status}
            try:
                ff_utils.patch_metadata(patch_data, obj_id=pairs['quality_metric'] ,connection=ff)
                success += 1
                print success
            except Exception as e:
                print e
                print
                patch_problem += 1
print success
print patch_problem

